# Microsoft Malware Prediction

## Información de cada variable

1. ~~**MachineIdentifier**~~: es el ID de la máquina y al ser único va a causar _overfitting_, ya que tiene 8921483 valores únicos, por lo que no aporta información, así que tenemos que quitarlo.

2. **ProductName**: es el programa de seguridad usado para defender el dispositivo, existen 6 categorías ('win8defender', 'mse', 'mseprerelease', 'scep', 'windowsintune' y 'fep'). Por ejemplo, 'win8defender' es el tipo de programa que más veces se repite, con 8826520 valores. El programa 'win8defender' de Windows Defender, usa protección en tiempo real para analizar todo lo que de descarga o lo que se ejecuta en el dispositivo. Por tanto, es un método de prevención y de eliminación de software espía en Microsoft Windows, que ya viene instalado y activado en los equipos con Windows 7, Windows 8 y Windows 10 (https://computerhoy.com/reportajes/tecnologia/5-motivos-utilizar-windows-defender-como-antivirus-no-hacerlo-308231).
  
> `train['ProductName'].value_counts()`

~~~
win8defender     8826520
mse                94873
mseprerelease         53
scep                  22
windowsintune          8
fep                    7
~~~

3. **EngineVersion**: indica la información de la versión del motor del programa de seguridad. La página de Windows Security About muestra Antimalware Client Version (Windows Defender version), Engine Version (Scanning Engine), Antivirus Version (Virus definitions), y Antispyware version (Spyware definitions), como se puede ver en https://www.bleepingcomputer.com/tutorials/how-to-find-windows-defender-version-number-installed-in-windows-10/. Hay 70 valores únicos, los cinco más comunes son:

> `train['EngineVersion'].value_counts()`

~~~
1.1.15200.1    3845067
1.1.15100.1    3675915
1.1.15000.2     265218
1.1.14901.4     212408
1.1.14600.4     160585
...
~~~

<img src="imagenes/EngineVersion.png" width=600 height=600 />

4. **AppVersion**: informa de la versión del cliente antimalware (versión de Windows Defender), como se ve en la anterior imagen. No presenta casi correlación con `EngineVersion`, `AvSigVersion` y `HasDetections` (todas entorno a 0.1). Las 5 versiones más comunes son:

> `train['AppVersion'].value_counts()`

~~~
4.18.1807.18075     5139224
4.18.1806.18062      850929
4.12.16299.15        359871
4.10.209.0           272455
4.13.17134.1         257270
...
~~~

5. **~~AvSigVersion~~**: información acerca del programa de seguridad, tiene 8531 valores únicos y añade más información redundante del mismo tipo que las variables `EngineVersion` y `AppVersion`. Además, se observa como tiene una alta correlación con `EngineVersion` (0.83), por lo que considero prescindir de ella. Las 5 versiones más comunes son:

> `train['AvSigVersion'].value_counts()`

~~~
1.273.1420.0         102317
1.263.48.0            98024
1.275.1140.0          97232
1.275.727.0           92448
1.273.371.0           86967
...
~~~

~~~python
# Veamos la correlación entre la variable categórica con la variable EngineVersion

X = train['EngineVersion']
y = train['AvSigVersion']

aux.corr_categorical(X,y) 
> 0.8389188441288544
~~~

~~~python
# Veamos la correlación entre la variable categórica con la variable AppVersion

X = train['AppVersion']
y = train['AvSigVersion']

aux.corr_categorical(X,y) 
> 0.16826741826532388
~~~

6. **IsBeta**: indica si el programa de seguridad es una beta o no. Es interesante ver esta variable de acuerdo a la variable target `HasDetections`, ya que se observa como la mayoría no son beta, lo que es lógico. Un software beta, a lo que también se conoce como beta a secas, es un programa que se encuentra cerca de su fase final de desarrollo y que puede ser funcional, pero que carece de los retoques finales que dan forma al producto terminado. Además, existe casi la misma distribución para dispositivos donde se ha detectado malware y donde no. 

> `train['IsBeta'].value_counts()`

~~~
0    8921416
1         67
~~~

> `train[['IsBeta', 'HasDetections']].groupby(['IsBeta', 'HasDetections']).size()`

~~~
IsBeta  HasDetections
0       0                4462557
        1                4458859
1       0                     34
        1                     33
~~~

7. **RtpStateBitfield**: no viene información en Kaggle de esta variable, pero lo más probable es que sea estado RTP (habilitado o deshabilitado), es decir, estado de protección en tiempo real, como se dice en https://docs.microsoft.com/en-us/windows/security/threat-protection/windows-defender-antivirus/troubleshoot-windows-defender-antivirus. No obstante, en este caso, debería ser una variable binaria y entera (aunque se encuentre el valor 0), pero pienso que es una variable que aporta información y tiene pocos missings (32318 - 0.36%).

> `train['RtpStateBitfield'].value_counts()`

~~~
7.0     8651487
0.0      190701
NA        32318
8.0       21974
5.0       20328
3.0        3029
1.0        1625
35.0         21
~~~

8. **IsSxsPassiveMode**: no viene información en Kaggle, pero en Internet se dice que es un modo de operación activo/pasivo para Windows Defender, es decir, si existe otro antivirus primario de terceros en el sistema, el Windows Defender ingresa al modo pasivo. El modo pasivo obviamente ofrece una funcionalidad reducida (http://techgenix.com/stick-with-windows-defender/). Los datos muestran que una máquina que está en modo pasivo tiene una mayor prevalencia de malwares (97768).

> `train['IsSxsPassiveMode'].value_counts()`

~~~
0    8766840
1     154643
~~~

> `train[['IsSxsPassiveMode', 'HasDetections']].groupby(['IsSxsPassiveMode', 'HasDetections']).size()`

~~~
IsSxsPassiveMode  HasDetections
0                 0                4364823
                  1                4402017
1                 0                  97768
                  1                  56875
~~~

9. ~~**DefaultBrowsersIdentifier**~~: es el ID para el navegador predeterminado de la máquina. Si juntamos que es un identificador con 2017 valores únicos y que tiene un 95.14% de NA (8488045), se opta por eliminarla.

10. **~~AVProductStatesIdentifier~~**: es el ID para la configuración específica del software antivirus de un usuario, tiene los valores en coma flotante y 28970 valores únicos, donde hay 36221 missings (0.41%). Se obtiene que está correlacionada de forma negativa con `AVProductsInstalled`. Las cinco primeros son:

> `train['AVProductStatesIdentifier'].value_counts()`

~~~
53447.0    5824565
7945.0      475897
47238.0     327656
62773.0     266764
46413.0     112878
...
~~~

~~~python
# Veamos la correlación con AVProductsInstalled y AVProductsEnabled

aux.plot_corr(aux.corr(train, ['AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled']))
~~~

<img src="imagenes/AVProductStatesIdentifier.png" width=500 height=500 />

11. **AVProductsInstalled**: no viene información en Kaggle de esta variable, pero parece que es el número de productos antivirus instalados, en donde el 90% de las máquinas tienen 1-2 productos instalados. Tiene 36221 missings (0.41%).

> `train['AVProductsInstalled'].value_counts()`

~~~
1.0    6208893
2.0    2459008
3.0     208103
NA       36221
4.0       8757
5.0        471
6.0         28
7.0          1
0.0          1
~~~

12. **AVProductsEnabled**: no viene información en Kaggle de esta variable, pero puede ser el número de productos antivirus instalados que están habilitados. Es interesante que esté está variable, ya que un antivirus instalado no tiene porqué estar habilitado. Tiene 36221 missings (0.41%).

> `train['AVProductsEnabled'].value_counts()`

~~~
1.0    8654101
2.0     198652
NA       36221
0.0      25958
3.0       6075
4.0        453
5.0         23
~~~

13. **HasTpm**: esta variable indica VERDADERO si la máquina tiene TPM, es decir, un Trusted Platform Module (TPM) que es un chip especializado en un dispositivo de punto final que almacena las claves de cifrado RSA específicas del sistema host para la autenticación de hardware. Cada chip TPM contiene un par de claves RSA llamado clave de respaldo (Endorsement Key - EK), en donde, el par se mantiene dentro del chip y el software no puede acceder a él. 

> `train['HasTpm'].value_counts()`

~~~
1    8814167
0     107316
~~~

14. **CountryIdentifier**: indica el ID del país en el que se encuentra la máquina (https://en.wikipedia.org/wiki/List_of_countries_by_United_Nations_geoscheme). Tiene 222 valores únicos. Por ejemplo, si se trata de códigos de país exactos, Austria (43) tiene el mayor número de filas en este conjunto de datos, mientras que EE. UU. (001) tiene solo el 2%. Los más comunes son:

> `train['CountryIdentifier'].value_counts()`

~~~
43     397172
29     347991
141    333411
93     283625
171    280572
...
~~~

~~~python
# Veamos la correlación con las variables ID de regiones que hay

aux.plot_corr(aux.corr(train, ['CountryIdentifier', 'GeoNameIdentifier', 'CityIdentifier',
                              'OrganizationIdentifier', 'LocaleEnglishNameIdentifier']))
~~~

<img src="imagenes/CountryIdentifier.png" width=500 height=500 />

15. **CityIdentifier**: es la identificación de la ciudad en la que se encuentra la máquina, tiene 107366 ciudades únicas y algunos missings (325409 - 3.65%). Los valores más comunes son:

> `train['CityIdentifier'].value_counts()`

~~~
NA         325409
130775.0    94812
16668.0     84780
82373.0     83312
10222.0     71814
61668.0     66845
...
~~~

16. **OrganizationIdentifier**: indica el ID de la organización a la que pertenece la máquina, el ID de la organización se asigna tanto para compañías específicas como industrias amplias. Hay 49 organizaciones únicas, donde el 50% de las máquinas están bajo una organización, otro 25% no clasificado. Tiene 2751518 missings (30.84%). Los más comunes son:
  
> `train['OrganizationIdentifier'].value_counts()`

~~~
27.0    4196457
NA      2751518
18.0    1764175
48.0      63845
50.0      45502
... 
~~~

17. **~~GeoNameIdentifier~~**: es el ID para la región geográfica en la que se encuentra una máquina, tiene 292 valores distintos, pero ya tenemos dos variables `CountryIdentifier` y `CityIdentifier`, que indican la localización de la máquina. Además, `CountryIdentifier` tiene 202 valores distintos, pero en la correlación entre `CityIdentifier`, `CountryIdentifier` y `GeoNameIdentifier`, con la variable target (`HasDetections`), tenemos un un valor de 0.6 de la correlación Pearson entre las variables `CountryIdentifier` y `GeoNameIdentifier`. Además, en Kaggle se dice que *The `CountryIdentifier` is highly associated with `GeoNameIdentifier`, `LocaleEnglishNameIdentifier`, `Census_OSInstallLanguageIdentifier`, `Census_OSUILocaleIdentifier`, and `Wdft_RegionIdentifier` as seen in the below correlation plot*. Tiene 213 missings (0.00%).

~~~python
# Veamos la correlación con esas variables

aux.plot_corr(aux.corr(train, ['CountryIdentifier', 'GeoNameIdentifier', 'LocaleEnglishNameIdentifier',
                              'Census_OSInstallLanguageIdentifier', 'Census_OSUILocaleIdentifier']))
~~~

<img src="imagenes/GeoNameIdentifier.png" width=500 height=500 />

18. **LocaleEnglishNameIdentifier**: es el nombre en inglés del ID de configuración regional del usuario actual. Tiene una correlación baja con `CountryIdentifier` (0.05). Por ejempo, si `CountryIdentifier` = 43 es `GeoNameIdentifier` = 53 es `LocaleEnglishNameIdentifier` = 42. (https://www.kaggle.com/c/microsoft-malware-prediction/discussion/78873). La variable contiene 276 valores únicos, es importante saber que "una localidad no es un idioma ni un país, el mismo idioma se puede hablar en varios países (a menudo con diferencias sutiles) y un solo país puede hablar varios idiomas. Por lo tanto, una localidad es un área donde se habla un idioma en particular que puede (o no) alinearse con los límites geográficos y/o políticos" (https://ss64.com/locale.html).

19. **Platform**: indica el nombre de la plataforma (propiedades relacionadas con el sistema operativo y el procesador).

> `train['Platform'].value_counts()`

~~~
windows10      8618715
windows8        194508
windows7         93889
windows2016      14371
~~~

20. **Processor**: es la arquitectura del sistema operativo instalado. El paquete X86 es para Windows de 32 bits y el paquete de X64 es para Windows de 64 bits. De ARM64 podemos decir que es una evolución de la arquitectura ARM, compatible con el procesamiento de datos de 64 bits y otorga una mayor potencia computacional, y poco a poco se está convirtiendo en el estándar en la mayoría de móviles modernos (https://www.xatakandroid.com/aplicaciones-android/arm-arm64-x86-dpi-como-saber-que-apk-descargarte-para-tu-movil).

> `train['Processor'].value_counts()`

~~~
x64      8105435
x86       815702
arm64        346
~~~

21. **OsVer**: indica la versión del sistema operativo actual. Las 5 versiones más usadas son:
    
> `train['OsVer'].value_counts()`

~~~
10.0.0.0        8632545
6.3.0.0          194447
6.1.1.0           93268
6.1.0.0             582
10.0.3.0            225
...
~~~

22. **OsBuild**: indica la compilación del sistema operativo actual. Tiene 76 valores únicos, de los cuales 5 forman la mayoría. La distribución de los 8 valores principales son:

> `train['OsBuild'].value_counts()`

~~~
17134    3915521
16299    2503681
15063     780270
14393     730819
10586     411606
10240     270192
9600      194508
7601       93306
...
~~~

23. **OsSuite**: indica la máscara del conjunto de productos para el sistema operativo actual, se observa como tiene una distribución muy sesgada.

> `train['OsSuite'].value_counts()`

~~~
768    5560661
256    3346251
272      12092
400        793
16         731
...
~~~

~~~python
# Veamos la correlación entre OsSuite y OsBuild

aux.plot_corr(aux.corr(train, ['OsSuite', 'OsBuild']))
~~~

<img src="imagenes/OsSuite.png" width=500 height=500 />

24. **OsPlatformSubRelease**: contiene la sub-versión de la plataforma SO (es menos sesgada que `OsSuite`). En el siguiente enlace https://support.lenovo.com/es/es/solutions/HT502786 se encuentran las versiones de Windows 10 con los nombres en clave de esta variable. 

> `train['OsPlatformSubRelease'].value_counts()`

~~~
rs4           3915526
rs3           2503681
rs2            780270
rs1            730819
th2            411606
th1            270192
windows8.1     194508
windows7        93889
prers5          20992
~~~

25. **~~OsBuildLab~~**: tiene información del compilador generada por el sistema operativo (17134.1.amd64fre.rs4_release.180410-1804). Tiene 21 missings (0.00%). Se observa cómo junta información de las variables `OsBuild`, `OsPlatformSubRelease` y `Processor`, entre otros, por lo que esta variable aporta información redundante, ya que dicha información está particiona en otras variables y considerado disponer de la información de forma separada:

~~~
OsBuildLab[0]             17134.1.amd64fre.rs4_release.180410-1804
OsPlatformSubRelease[0]                                        rs4
OsBuild[0]                                                   17134
Processor[0]                                                   x64
~~~

26. **SkuEdition**: esta variable tiene como objetivo definir el tipo de producto en el MSDN, con el fin de asignar un nombre 'SKU-Edition' que sea útil en los informes de población. Los tipos de productos válidos se definen en `%sdxroot%\data\windowseditions.xml`. Además, esta API se ha utilizado desde Vista y Server 2008, por lo que hay muchos tipos de productos que no se aplican a Windows 10. Es conveniente, pasar a minúsculas todas las etiquetas.

> `train['SkuEdition'].value_counts()`

~~~
Home               5514341
Pro                3224164
Invalid              78054
Education            40694
Enterprise           34357
Enterprise LTSB      20702
Cloud                 5589
Server                3582
...
~~~

27. **IsProtected**: es una variable calculada (a partir de Spynet Report's AV Products) que nos dice si una máquina está protegida. La variable puede tomar tres valores: VERDADERA si hay al menos un producto antivirus activo y actualizado ejecutándose en la máquina; FALSA si no hay un producto AV activo en esa máquina, o si el AV está activo, pero no recibe las últimas actualizaciones; y NA si no hay productos antivirus en el informe. Es importante saber que una máquina que no está protegida por un antivirus tiene más posibilidades de malware que una que está protegida. Tiene 36044 missings (0.40%).

> `train['IsProtected'].value_counts()`

~~~
1.0    8402282
0.0     483157
NA       36044
~~~

> `train[['IsProtected', 'HasDetections']].groupby(['IsProtected', 'HasDetections']).size()`

~~~
IsProtected  HasDetections
0.0          0                 298904
             1                 184253
1.0          0                4141184
             1                4261098
~~~

28. **~~AutoSampleOptIn~~**: en Kaggle pone que es el valor `SubmitSamplesConsent` pasado del servicio, disponible en CAMP 9+, pero en las búsquedas en Internet, no he localizado que es exactamente. Puesto que tiene solo 258 valores para una categoría, donde el 99.99% pertenecen a la otra, he optado por eliminarlo.

> `train['AutoSampleOptIn'].value_counts()`

~~~
0    8921225
1        258
~~~

29. **PuaMode**: indica si está el modo PUA habilitado desde el servicio (a función de protección de aplicaciones potencialmente no deseadas (PUA) en Windows Defender Antivirus puede identificar y bloquear la descarga e instalación de PUA en puntos finales de su red). Es importante saber que estas aplicaciones no se consideran virus, malware u otros tipos de amenazas, pero pueden realizar acciones en puntos finales que afectan negativamente su rendimiento o uso, también puede referirse a aplicaciones que se consideran que tienen una mala reputación (https://www.tenforums.com/tutorials/32236-enable-disable-windows-defender-pua-protection-windows-10-a.html). Tiene 8919174 missings (99.97%).
    
> `train['PuaMode'].value_counts()`

~~~
NA    8919174
on       2307
audit       2
~~~

30. **SMode**: este campo se establece en VERDADERO cuando se sabe que el dispositivo está en modo S (como en el modo S de Windows 10), donde solo se pueden instalar aplicaciones de la tienda Microsoft. Tiene 537759 missings (6.03%).

> `train['SMode'].value_counts()`

~~~
0.0    8379843
NA      537759
1.0       3881
~~~

31. **IeVerIdentifier**: no viene información en Kaggle sobre esta variable, pero esta variable dice que versión de Internet Explorer se está ejecutando en el dispositivo o máquina (https://docs.microsoft.com/en-us/windows/privacy/basic-level-windows-diagnostic-events-and-fields-1703#census-events). Tiene 303 valores únicos y 58894 missings (0.66%), los valores más frecuentes son:

> `train['IeVerIdentifier'].value_counts()`

~~~
137.0    3885842
117.0    1767931
108.0     474390
111.0     467828
98.0      354411
135.0     217458
53.0      204952
...
~~~


32. **SmartScreen**: es el valor de cadena habilitado para SmartScreen del registro. Esto se obtiene marcando en orden `HKLM\SOFTWARE\Policies\Microsoft\Windows\System\SmartScreenEnabled` y `HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\SmartScreenEnabled`. Si el valor existe pero está en blanco, el valor "ExistsNotSet" se envía por telemetría. Entonces, SmartScreen de Windows Defender ayuda a proteger a los usuarios de si intentan visitar sitios que anteriormente se informaron como sitios web de phishing o malware, o si un usuario intenta descargar archivos potencialmente maliciosos. Esto solo se aplica a Windows 10 y Windows 10 Mobile. Tiene 3177011 missings (35.61%), en donde es necesario pasar a minúscula todas las etiquetas, ya que se ve como *Off* y *off* dicen lo mismo. Los más comunes son:

> `train['SmartScreen'].value_counts()`

~~~
RequireAdmin    4316183
ExistsNotSet    1046183
NA              3177011
Off              186553
Warn             135483
Prompt            34533
Block             22533
off                1350
On                  731
...
~~~

33. **Firewall**: indica si el firewall está habilitado, siendo VERDADERO (1) para Windows 8.1 y superior. Un firewall (llamado también «cortafuegos»), es un sistema que permite proteger a una computadora o una red de computadoras de las intrusiones que provienen de una tercera red (generalmente de Internet). El firewall es un sistema que permite filtrar los paquetes de datos que andan por la red, puede ser un programa (software) o un equipo (hardware) que actúa como intermediario entre la red local (o la computadora local) y una o varias redes externas. Tiene 91350 missings (1.02%).

> `train['Firewall'].value_counts()`

~~~
1.0    8641014
0.0     189119
NA       91350
~~~

34. **UacLuaenable**: este atributo informa si el tipo de usuario "administrador en Admin Approval Mode" está deshabilitado o habilitado en UAC. El valor informado se obtiene leyendo la clave de acceso `HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\EnableLUA`. UAC significa User Access Control (Control de Acceso de Usuario). En el enlace https://www.reddit.com/r/Windows10/comments/8dxzcz/uac_do_you_want_to_allow_this_app_to_make_changes/, hay una discusión acerca de esto. Tiene 10838 missings (0.12%).

> `train['UacLuaenable'].value_counts()`

~~~
1.0           8856517
0.0             53851
48.0              206
2.0                30
49.0               17
6357062.0          13
3.0                 6
5.0                 2
16777216.0          1
7798884.0           1
255.0               1
~~~    

35. **Census_MDC2FormFactor**: es una agrupación basada en una combinación de características de hardware de nivel de Device Census. La lógica utilizada para definir el factor, se basa en los estándares comerciales e industriales y se alinea con la forma en que las personas piensan acerca de su dispositivo. (ejemplos: Smartphone, Small Tablet, All in One, Convertible...). Tiene 623 missings (0.01%) y debemos pasar las categorías a minúsculas.

> `train['Census_MDC2FormFactor'].value_counts()`

~~~
Notebook        5723319
Desktop         1951086
Convertible      405378
Detachable       298233
AllInOne         292077
PCOther          139955
...
~~~

~~~python
# Veamos la correlación con la variable Census_ChassisTypeName

X = train['Census_ChassisTypeName']
y = train['Census_MDC2FormFactor']

aux.corr_categorical(X,y) 
> 0.45525073155024554
~~~

~~~python
# Veamos la correlación con la variable Census_PowerPlatformRoleName

X = train['Census_PowerPlatformRoleName']
y = train['Census_MDC2FormFactor']

aux.corr_categorical(X,y) 
> 0.4758266883436838
~~~

36. **Census_DeviceFamily**: indica el tipo de dispositivo para el que está destinada una edición del sistema operativo (AKA DeviceClass). Tiene una alta proporción para Windows Desktop y baja para Windows Server y Windows. De igual manera, deberemos pasar a minúsculas.
    
> `train['Census_DeviceFamily'].value_counts()`

~~~
Windows.Desktop    8907053
Windows.Server       14410
Windows                 20
~~~

~~~python
# Veamos la correlación con la variable Census_DeviceFamily

X = train['Census_MDC2FormFactor']
y = train['Census_DeviceFamily']

aux.corr_categorical(X,y) 
> 0.5937915311557042
~~~

37. **Census_OEMNameIdentifier**: no viene información sobre esta variable en Kaggle y no he encontrado nada acerca en Google. Tiene 95478 missings (1.07%).
    
> `train['Census_OEMNameIdentifier'].value_counts()`    

~~~
2668.0    1287275
2102.0    1038567
1443.0     949531
2206.0     924349
585.0      895452
525.0      842364
4589.0     310701
4730.0     304102
1980.0     285600
~~~

38. **Census_OEMModelIdentifier**: no viene información de esta variable en Kaggle y no he encontrado nada acerca en Google. Tiene 102233 missings (1.15%).
    
> `train['Census_OEMModelIdentifier'].value_counts()`

~~~
313586.0    304782
242491.0    263382
317701.0    139035
317708.0    115257
228975.0     79878
...
~~~   

~~~python
# Veamos la correlación con la variable Census_OEMNameIdentifier

aux.plot_corr(aux.corr(train, ['Census_OEMModelIdentifier', 'Census_OEMNameIdentifier']))
~~~

<img src="imagenes/Census_OEMModelIdentifier.png" width=500 height=500 />

39. **Census_ProcessorCoreCount**: es el número de núcleos lógicos en el procesador. Tiene 45 valores únicos, donde 4 procesadores son los más comunes. Tiene 41306	missings (0.46%).

> `train['Census_ProcessorCoreCount'].value_counts()`

~~~
4.0      5430193
2.0      2311969
8.0       865004
12.0       92702
1.0        70390
6.0        69910
16.0       18551
...
~~~

40. **Census_ProcessorManufacturerIdentifier**: no viene información sobre esta variable en Kaggle y no he encontrado nada acerca en Google. Tiene 41306 missings (0.46%).
 
> `train['Census_ProcessorManufacturerIdentifier'].value_counts()`

~~~
5.0     7839318
1.0     1040292
10.0        339
3.0         218
9.0           1
7.0           1
4.0           1
~~~

41. **~~Census_ProcessorModelIdentifier~~**: no viene información en Kaggle sobre esta variable. Tiene 41306	missings (0.46%), 3428 valores únicos y presenta una alta correlación con la variable `Census_ProcessorManufacturerIdentifier` de 0.8, por lo se quita.

> `train['Census_ProcessorModelIdentifier'].value_counts()`

~~~
2697.0    289283
1998.0    267397
2660.0    191392
2373.0    175407
1992.0    171728
2382.0    170651
2640.0    153719
...
~~~

~~~python
# Veamos la correlación con la variable Census_ProcessorManufacturerIdentifier

aux.plot_corr(aux.corr(train, ['Census_ProcessorManufacturerIdentifier',
                               'Census_ProcessorModelIdentifier']))
~~~
                               
<img src="imagenes/Census_ProcessorModelIdentifier.png" width=500 height=500 />    

42. **~~Census_ProcessorClass~~**: da una clasificación de procesadores en alta/media/baja. Esta variable se usaba inicialmente para precios de SKU pero ya no se mantiene ni se actualiza en el pograma de seguridad. Además, cuenta con 8884852 missings (99.59%) por lo que se opta por eliminar, ya que a partir de `SkuEdition` y `Processor` se puede deducir a que gama pertenece el procesador.

> `train['Census_ProcessorClass'].value_counts()`

~~~
NA    8884852
mid     20914
low      9621
high     6096
~~~

43. **Census_PrimaryDiskTotalCapacity**: es la cantidad de espacio en disco en el disco primario de la máquina en MB. Tiene 5735 valores únicos (53016 missings - 0.59%). Está en MB, estaría interesante ver si mejora al pasarlo a GB. Los tamaños de disco más populares son 500GB y 1TB.

> `train['Census_PrimaryDiskTotalCapacity'].value_counts()`

~~~
476940.0     2841530
953869.0     2175780
305245.0      474616
122104.0      469060
...
~~~

44. **Census_PrimaryDiskTypeName**: es el nombre descriptivo del tipo de disco primario (HDD o SSD). Se ve como HDD es mucho más popular que SSD y que UNKNOWN y Unspecified, denotan lo mismo (se deberá pasar a minúscula). Tiene 12844 missings (0.14%).

> `train['Census_PrimaryDiskTypeName'].value_counts()`

~~~
HDD            5806804
SSD            2466808
UNKNOWN         358251
Unspecified     276776
NA               12844
~~~

45. **Census_SystemVolumeTotalCapacity**: es el tamaño de la partición en la que está instalado el volumen del sistema en MB (estaría interesante pasarlos a GB). Existen 536848 tamaños de volumen de sistema únicos (53002 missings - 0.59%).

> `train['Census_SystemVolumeTotalCapacity'].value_counts()`

~~~
28542.0     51998
926992.0    50430
476389.0    44435
953253.0    41572
102400.0    41257
476324.0    40925
...
~~~

46. **Census_HasOpticalDiskDrive**: esta variable es VERDADERO si indica que la máquina tiene una unidad de disco óptico (CD/DVD). La mayoría de los sistemas no tienen una unidad de disco. Por lo que, esta variable podría proporcionar una idea de la antigüedad del hardware, ya que el hardware más antiguo podría correlacionarse con tasas de malware más altas. 

> `train['Census_HasOpticalDiskDrive'].value_counts()`

~~~
0    8232858
1     688625
~~~

47. **Census_TotalPhysicalRAM**: da información acerca de la RAM física en MB. Como están en MB, sería interesante convertirlo a GB dividiendo por 1024 y luego crear un grupo de frecuencias. La mayoría de las RAM tienen entre 2 y 8 GB de RAM. Tiene 80533 missings (0.90%).

> `train['Census_TotalPhysicalRAM'].value_counts()`

~~~
4096.0      4094512
8192.0      2196505
2048.0      1097474
16384.0      531558
6144.0       398671
12288.0      159894
...
~~~

48. **Census_ChassisTypeName**: define qué tipo de hardware tiene la máquina (tiene valores numéricos y no numéricos). Los más populares no parecen ser mutuamente excluyentes. Además, tanto Notebook como Portable podrían usarse indistintamente, ya que un Notebook es un Portable, pero a diferencia de una Laptop, un Notebook posee un (ligeramente) menor rendimiento que un Laptop (https://difiere.com/diferencia-laptop-y-notebook/) y también debe pasar a minúscula. Los 5 tipos más comunes son:

> `train['Census_ChassisTypeName'].value_counts()`

~~~
Notebook               5248812
Desktop                1872125
Laptop                  685581
Portable                360903
AllinOne                204295
...
~~~

49. **Census_InternalPrimaryDiagonalDisplaySizeInInches**: da la longitud diagonal física en pulgadas de la pantalla principal. Contiene 785 tamaños de pantalla únicos. Esta variable se puede usar junto con `Census_ChassisTypeName` para diferenciar aún más a los dispositivos.  Lo más probable es que las pantallas más grandes tengan una resolución más alta. Tiene 47134 missings	(0.53%) y los más comunes son:

> `train['Census_InternalPrimaryDiagonalDisplaySizeInInches'].value_counts()`

~~~
15.500000    3047431
13.900000     952078
14.000000     542450
11.600000     319376
21.500000     275337
...
~~~

50. **Census_InternalPrimaryDisplayResolutionHorizontal**: devuelve el número de píxeles en la dirección horizontal de la pantalla interna (número de líneas verticales). Tiene 2180 valores únicos, en donde, los más populares son 1366 y 1920 de resolución. Tiene 46986 missings (0.53%). Estos valores pueden indicar la edad del dispositivo. Las resoluciones más altas probablemente significan que los dispositivos son relativamente nuevos, en comparación con el resto (https://calibracionhd.com/resoluciones-de-pantalla/). Además, tiene una correlación muy alta con `Census_InternalPrimaryDisplayResolutionVertical` (0.9).

> `train['Census_InternalPrimaryDisplayResolutionHorizontal'].value_counts()`

~~~
1366.0    4515064
1920.0    2220648
1280.0     527430
1600.0     501288
1024.0     342620
...
~~~

~~~python
# Veamos la correlación con las demás variables de la resolución

aux.plot_corr(aux.corr(train, ['Census_InternalPrimaryDiagonalDisplaySizeInInches',
                               'Census_InternalPrimaryDisplayResolutionHorizontal',
                               'Census_InternalPrimaryDisplayResolutionVertical']))
~~~

<img src="imagenes/Census_InternalPrimaryDisplayResolutionHorizontal.png" width=500 height=500 />    

51. **~~Census_InternalPrimaryDisplayResolutionVertical~~**: devuelve el número de píxeles en la dirección vertical de la pantalla interna (número de líneas horizontales). Tiene 46986 missings (0.53%). Como tiene una correlación muy alta con `Census_InternalPrimaryDisplayResolutionHorizontal` (0.9), puesto que la resolución horizontal y vertical son dependientes, se opta por prescindir de ella, ya que se puede usar la medida de la resolución horizontal para obtener la de la resolución vertical. Tiene 1560 valores únicos, los más comunes son:
    
> `train['Census_InternalPrimaryDisplayResolutionVertical'].value_counts()`

~~~
768.0     4973621
1080.0    2148402
900.0      655155
800.0      262058
1024.0     186322
...
~~~

<img src="imagenes/resolucion.png" width=500 height=500 />   

52. **Census_PowerPlatformRoleName**: indica el perfil de administración de energía preferido por el OEM. Este valor ayuda a identificar el factor de forma básico del dispositivo, tiene una correlación intermedia con las variables `Census_ChassisTypeName` y `Census_MDC2FormFactor`. Tiene 55 missings (0.00%), se debe pasar a minúsculas y combinar Unspecified con UNKNOWN.

> `train['Census_PowerPlatformRoleName'].value_counts()`

~~~
Mobile               6182908
Desktop              2066620
Slate                 492537
Workstation           109683
SOHOServer             37841
UNKNOWN                20628
EnterpriseServer        7094
AppliancePC             4015
PerformanceServer         97
Unspecified                5
~~~

~~~python
# Veamos la correlación con la variable Census_ChassisTypeName

X = train['Census_ChassisTypeName']
y = train['Census_PowerPlatformRoleName']

aux.corr_categorical(X,y) 
> 0.46746623939526544
~~~

~~~python
# Veamos la correlación con la variable Census_MDC2FormFactor

X = train['Census_MDC2FormFactor']
y = train['Census_PowerPlatformRoleName']

aux.corr_categorical(X,y) 
> 0.4758266883436838
~~~

53. **~~Census_InternalBatteryType~~**: no viene información en Kaggle de esta variable. Esta variable tiene 6338429 missings (71.05%). Aparecen muchas categorías con nombres inconsistentes, como por ejemplo: '#', 'lion', '4cel', 'l & # TAB #'. La mayoría de los dispositivos están entre las 10 primeras categorías. Algunas de estas etiquetas parecen similares, por ejemplo: lion, li-i y liio podrían ser marcadores de posición para baterías de iones de litio, pero debido a estas inconsistencias se opta por prescindir de ella.
    
> `train['Census_InternalBatteryType'].value_counts()`

~~~
lion        2028256
li-i         245617
#            183998
lip           62099
liio          32635
li p           8383
li             6708
nimh           4614
real           2744
...
~~~

54. **Census_InternalBatteryNumberOfCharges**: no viene información en Kaggle sobre esta variable, pero se puede suponer que es el número de ciclos de batería. Si los ciclos de la batería se establecen en cero, se puede suponer que estos dispositivos se encuentran en el primer ciclo de carga de la batería. Se observa como el 56% de las máquinas están en su primer ciclo de carga de la batería o no funcionan con batería, puesto que al iniciar un ordenador por primera vez, éste debe de estar conectado a la corriente. Tiene 268755 missings (3.01%).
    
> `train['Census_InternalBatteryNumberOfCharges'].value_counts()`

~~~
0.000000e+00    5053404
4.294967e+09    2252338
1.000000e+00      53810
...
~~~

55. **~~Census_OSVersion~~**: indica la versión del SO de forma numérica (10.0.17134.228). Contiene 469 valores únicos. Se observa como es una variable compuesta por otras (`OsVer`, `Census_OSBuildRevision`, `Census_OSBuildNumber`), por lo que se puede eliminar, ya que se prefiere tener la información de forma separada.
    
> `train['Census_OSVersion'].value_counts()`

~~~
10.0.17134.228      1413627
10.0.17134.165       899711
10.0.16299.431       546546
10.0.17134.285       470280
10.0.16299.547       346853
10.0.17134.112       346410
...
~~~  

> `train['Census_OSVersion'].head()`
 
~~~
0    10.0.17134.165
1      10.0.17134.1
2    10.0.17134.165
~~~

> `train['OsVer'].head()`    

~~~
0    10.0.0.0
1    10.0.0.0
2    10.0.0.0
~~~

> `train['Census_OSBuildRevision'].head()`

~~~
0    165
1      1
2    165
~~~

> `train['Census_OSBuildNumber'].head()`

~~~
0    17134
1    17134
2    17134
~~~

~~~python
# Veamos la correlación con la variable OsVer

X = train['OsVer']
y = train['Census_OSVersion']

aux.corr_categorical(X,y) 
> 0.0846079076423891
~~~

~~~python
# Veamos la correlación con la variable Platform

X = train['Platform']
y = train['Census_OSVersion']

aux.corr_categorical(X,y) 
> 0.5320779917302222
~~~

~~~python
# Veamos la correlación con la variable Census_OSBuildRevision

X = train['Census_OSVersion']
y = train['Census_OSBuildRevision']

aux.corr_categorical(X,y) 
> 0.9999891195324985
~~~

56. **~~Census_OSArchitecture~~**: es la arquitectura en la que se basa el sistema operativo. Ya hay una variable `Processor` que nos da esta información (alta correlación de 0.99), por lo que se elimina.
    
> `train['Census_OSArchitecture'].value_counts()`

~~~
amd64    8105885
x86       815252
arm64        346
~~~

~~~python
# Veamos la correlación con la variable Processor

X = train['Census_OSArchitecture']
y = train['Processor']

aux.corr_categorical(X,y) 
> 0.9947207910769047
~~~

57. **~~Census_OSBranch~~**: es la rama del sistema operativo extraída de `OsVersionFull`, tiene 32 valores únicos. Da más información acerca del tipo de plataforma que es. Como amplía a la variable `OsPlatformSubRelease`, es posible prescindir de ella, puesto que es una jerarquía de la misma y tiene una alta correlación con dicha variable. Entonces, se opta por eliminarla, puesto que los primeros valores hacen referencia a los mismos que en `OsVersionFull`.

> `train['Census_OSBranch'].value_counts()`

~~~
rs4_release                  4009158
rs3_release                  1237321
rs3_release_svc_escrow       1199767
rs2_release                   797066
rs1_release                   785534
...
~~~

> `train['OsPlatformSubRelease'].value_counts()`

~~~
rs4           3915526
rs3           2503681
rs2            780270
rs1            730819
...
~~~

> `train[['OsPlatformSubRelease','Census_OSBranch']].groupby(['OsPlatformSubRelease','Census_OSBranch']).size()`

~~~python
# Veamos la correlación que tiene con OsPlatformSubRelease

X = train['Census_OSBranch']
y = train['OsPlatformSubRelease']

aux.corr_categorical(X,y) 
> 0.8522534071924249
~~~

58. **~~Census_OSBuildNumber~~**: indica número de compilación del sistema operativo extraído de `OsVersionFull`. Tiene 165 valores únicos y una alta correlación con la variable `OsBuild`, por lo que se elimina.
    
> `train['Census_OSBuildNumber'].value_counts()`    

~~~
17134    4008881
16299    2443249
15063     797049
14393     785450
10586     593527
...
~~~

~~~python
# Veamos la correlación que tiene con OsBuild

aux.plot_corr(aux.corr(train, ['Census_OSBuildNumber','OsBuild']))
~~~

<img src="imagenes/Census_OSBuildNumber.png" width=500 height=500 />    

59. **Census_OSBuildRevision**: indica el valor de compilación del SO extraída de `OsVersionFull` (`OsBuildRevision` = 1000 o 16458). Se observa como coincide con el final de la variable `Census_OSVersion`, por lo que podemos prescindir de `Census_OSVersion`. Tiene 285 valores únicos, los más comunes son: 
    
> `train['Census_OSBuildRevision'].value_counts()`
 
~~~
228      1413633
165       899712
431       546548
285       470280
547       346853
112       346488
...
~~~

> `train['Census_OSVersion'].value_counts()`

~~~
10.0.17134.228      1413627
10.0.17134.165       899711
10.0.16299.431       546546
10.0.17134.285       470280
10.0.16299.547       346853
10.0.17134.112       346410
...
~~~   

~~~python
# Veamos la correlación con la variable OsBuild

X = train['OsBuild']
y = train['Census_OSBuildRevision']

aux.corr_categorical(X,y) 
> 0.31649314140803825
~~~

~~~python
# Veamos la correlación con la variable Census_OSBuildRevision

X = train['Census_OSVersion']
y = train['Census_OSBuildRevision']

aux.corr_categorical(X,y) 
> 0.9999891195324985
~~~

60. **Census_OSEdition**: indica la edición del sistema operativo actual, procedente de `HKLM\Software\Microsoft\Windows NT\CurrentVersion@EditionID` en el registro. Tiene 33 valores únicos, los cuales deberán pasarse a minúsculas. Esta columna también puede estar vinculada al tipo de licencia (`Census_ActivationChannel`). Los 5 más comunes son:
    
> `train['Census_OSEdition'].value_counts()`   

~~~
Core                           3469991
Professional                   3130566
CoreSingleLanguage             1945461
CoreCountrySpecific             166100
ProfessionalEducation            56698
Education                        40704
Enterprise                       35603
ProfessionalN                    28341
...
~~~

~~~python
# Veamos la correlación con la variable Census_ActivationChannel
X = train['Census_ActivationChannel']
y = train['Census_OSEdition']

aux.corr_categorical(X,y) 
> 0.5131328802764941
~~~

61. **~~Census_OSSkuName~~**: indica el nombre descriptivo de la edición del sistema operativo (actualmente solo Windows). Tiene 30 valores únicos. Está muy sesgada por un gran número de: 'CORE', 'PROFESSIONAL', 'CORE_SINGLELANGUAGE' y 'CORE_COUNTRYSPECIFIC' y presenta una alta correlación con `Census_OSEdition`. De igual manera, tendremos que pasar a minúscula los valores.
    
> `train['Census_OSSkuName'].value_counts()`

~~~
CORE                            3469869
PROFESSIONAL                    3187913
CORE_SINGLELANGUAGE             1945133
CORE_COUNTRYSPECIFIC             165886
EDUCATION                         40827
ENTERPRISE                        35602
PROFESSIONAL_N                    28522
ENTERPRISE_S                      20022
...
~~~    

~~~python
# Veamos la correlación con la variable Census_OSEdition

X = train['Census_OSEdition']
y = train['Census_OSSkuName']

aux.corr_categorical(X,y) 
> 0.9050844970826697
~~~

62. **Census_OSInstallTypeName**: indica la descripción de qué instalación se utilizó en la máquina. Debemos convertir los valores en minúscula y arreglar inconsistencias ortográficas y/o gramaticales, puesto que he buscado si existe 'UUPUpgrade', y no he encontrado nada.

> `train['Census_OSInstallTypeName'].value_counts()`

~~~
UUPUpgrade        2608037
IBSClean          1650733
Update            1593308
Upgrade           1251559
Other              840121
Reset              649201
Refresh            205842
Clean               69073
CleanPCRefresh      53609
~~~

63. **Census_OSInstallLanguageIdentifier**: no viene información en Kaggle de esta variable, pero parece que está relacionada con el identificador del idioma para el SO. Tiene 39 valores únicos y 60084 missings (0.67%). Presenta una alta correlación con la variable `Census_OSUILocaleIdentifier`.
    
> `train['Census_OSInstallLanguageIdentifier'].value_counts()`  

~~~
8.0     3179262
9.0     1034201
7.0      512753
29.0     492267
14.0     432503
37.0     403190
...
~~~

~~~python
# Veamos la correlación con otras variables de este mismo tipo

aux.plot_corr(aux.corr(train, ['Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier',
                              'LocaleEnglishNameIdentifier', 'CountryIdentifier', 'CityIdentifier',
                              'OrganizationIdentifier', 'GeoNameIdentifier']))
~~~

<img src="imagenes/Census_OSInstallLanguageIdentifier.png" width=500 height=500 />    

64. **~~Census_OSUILocaleIdentifier~~**: no viene información en Kaggle de esta variable, tiene 147 valores únicos, en donde los valores que toman 31, 34 y 30 son los 3 principales identificadores locales. Presenta una alta correlación con la variable `Census_OSInstallLanguageIdentifier`, por lo que se opta por eliminar esta variable.

> `train['Census_OSUILocaleIdentifier'].value_counts()`

~~~
31     3170824
34     1040042
30      513995
125     498236
...
~~~

~~~python
# Veamos la correlación con la variable Census_OSInstallLanguageIdentifier

X = train['Census_OSUILocaleIdentifier']
y = train['Census_OSInstallLanguageIdentifier']

aux.corr_categorical(X,y) 
> 0.9911874845849825
~~~

65. **Census_OSWUAutoUpdateOptionsName**: indica el nombre descriptivo de la configuración de actualización automática de WindowsUpdate en la máquina. De igual manera, deberemos pasarlos a minúscula.
   
> `train['Census_OSWUAutoUpdateOptionsName'].value_counts()`

~~~
FullAuto                                 3954497
UNKNOWN                                  2519925
Notify                                   2034254
AutoInstallAndRebootAtMaintenanceTime     371475
Off                                        26961
DownloadNotify                             14371
~~~

66. **Census_IsPortableOperatingSystem**: indica si el sistema operativo se inicia y ejecuta a través de Windows-To-Go en una memoria USB, suponiendo que 0 es FALSO para la portabilidad, es decir, que no se ejecuta en una memoria USB.

> `train['Census_IsPortableOperatingSystem'].value_counts()`

~~~
0    8916619
1       4864
~~~

67. **Census_GenuineStateName**: es el nombre de OSGenuineStateID. Según parece, hace referencia a si la licencia para Windows es original (https://answers.microsoft.com/en-us/windows/forum/windows_7-winapps/genuine-windows7/04a6fe05-56ba-40cc-81d9-93d5dd7a4105, https://support.microsoft.com/en-us/help/4487266/activation-failures-and-not-genuine-notifications-on-vl-win-7-kms-clie). De igual manera, deberemos pasarlos a minúscula.

> `train['Census_GenuineStateName'].value_counts()`

~~~
IS_GENUINE         7877597
INVALID_LICENSE     801692
OFFLINE             228366
UNKNOWN              13826
TAMPERED                 2
~~~

68. **Census_ActivationChannel**: clave de licencia comercial o clave de licencia por volumen para una máquina (https://social.microsoft.com/Forums/en-US/44179f86-f8a6-4dc2-8692-b1637e72280b/windows-license-types-explained?forum=genuinewindows7). De igual manera, deberemos pasarlos a minúscula.

    - *Retail*: las claves de productos minoristas son lo que obtiene el cliente cuando compra un producto empaquetado  de Windows de un comerciante minorista o compra Windows en línea en la tienda de Microsoft.
    - *OEM SLP y COA SLP*: las claves de producto OEM SLP y COA SLP son emitidas por grandes fabricantes de computadoras y utilizan la tecnología SLP (System Locked Pre-installation) para vincular la licencia a la placa base original a través del BIOS y el software. 
    - *OEM System Builder*: las claves de producto son para el uso de constructores de sistemas más pequeños, talleres informáticos, consultores y otros que proporcionan computadoras y servicios a sus clientes. La licencia de System Builder define a un creador de sistemas como "un fabricante de equipos originales, un ensamblador, un restaurador o un preinstalador de software que vende los sistemas del dliente a un tercero".
    - *GVLK* (clave de licencia por volumen de grupo): es solo un acrónimo de una versión de licencia por volumen de Windows. Principalmente disponible para grandes empresas e instituciones gubernamentales y viene con soporte premium. GLVK también necesita un KMS (servicio de administración de claves) para la implementación.
    - *OEM NON-SLP*: significa claves de licencia de un OEM pero no está vinculado a un System Locked Pre-installation. Las claves OEM no se pueden revender y no se pueden transferir a otra computadora. Sin embargo, pueden transferirse con la computadora si la computadora se transfiere a una nueva propiedad.
    - *Volume MAK*: las claves de producto KMS Client y Volume MAK son claves de licencia por volumen que no se pueden revender. Son emitidos por organizaciones para su uso en equipos cliente asociados de alguna manera con la organización. Las claves de licencia por volumen no se pueden transferir con la computadora si la computadora cambia de propietario.

> `train['Census_ActivationChannel'].value_counts()`

~~~
Retail            4727589
OEM:DM            3413350
Volume:GVLK        450954
OEM:NONSLP         317980
Volume:MAK           8028
Retail:TB:Eval       3582
~~~

69. **~~Census_IsFlightingInternal~~**: en el contexto de Windows Defender, 'flighting' significa poner a disposición nuevas funciones de desarrollo lo antes posible, durante el ciclo de desarrollo. Esto no se refiere a un lanzamiento público. Probablemente, hace referencia a desarollos internos referentes a la comunidad de Windows Insider (https://docs.microsoft.com/en-us/windows/deployment/update/waas-overview). Tiene 7408759 missings (83.04%) y 21 observaciones solo para una categoría, por lo que se elimina.

> `train['Census_IsFlightingInternal'].value_counts()`

~~~
NA     7408759
0.0    1512703
1.0         21
~~~

70. **Census_IsFlightsDisabled**: indica si la variable está en 'flighting'. Se ve como la mayoría de los dispositivos tienen habilitado el 'flighting'. Lo que significa que la mayoría de estas máquinas son del programa Windows Insider. Con Windows 10, esta función permite a los Insiders de Windows consumir e implementar código de preproducción en sus máquinas de prueba, obteniendo visibilidad temprana en la próxima compilación. En el pasado, cuando Microsoft desarrollaba nuevas versiones de Windows, generalmente lanzaba avances técnicos cerca del final del proceso, cuando Windows estaba casi listo para su lanzamiento. Con Windows 10, las nuevas características se entregarán a la comunidad de Windows Insider lo antes posible, durante el ciclo de desarrollo, a través de un proceso llamado 'flighting', para que las organizaciones puedan ver exactamente lo que Microsoft está desarrollando y comenzar sus pruebas lo antes posible (https://docs.microsoft.com/en-us/windows/deployment/update/waas-overview). Tiene 160523 missings (1.80%).

> `train['Census_IsFlightsDisabled'].value_counts()`

~~~
0.0    8760872
1.0         88
~~~

71. **Census_FlightRing**: indica el anillo para el que el usuario del dispositivo desea recibir 'flights'. De igual manera, deberemos pasarlos a minúscula.

> `train['Census_FlightRing'].value_counts()`

~~~
Retail      8355679
NOT_SET      287803
Unknown      243438
WIS           10648
WIF           10322
RP             9860
Disabled       3722
OSG               7
Canary            3
Invalid           1
~~~

72. **~~Census_ThresholdOptIn~~**: no viene información en Kaggle sobre esta variable. Tiene 5667325 missings (63.52%) y 816 valores para una categoría, siendo predominante el 0, por lo se opta por prescindir de ella.
    
> `train['Census_ThresholdOptIn'].value_counts()`

~~~
NA     5667325
0.0    3253342
1.0        816
~~~

73. **Census_FirmwareManufacturerIdentifier**: no viene información en Kaggle de esta variable, pero es el identificador de fabricantes del firmware. El firmware, también conocido como soporte lógico inalterable, es el programa básico que controla los circuitos electrónicos de cualquier dispositivo (https://www.xataka.com/basics/que-firmware-que-se-diferencia-drivers). Tiene 712 valores únicos, con la mayoría concentrada en los 5 principales (tiene 183257 missings - 2.05%).

> `train['Census_FirmwareManufacturerIdentifier'].value_counts()`

~~~
142.0     2699078
628.0     1229140
554.0     1175137
355.0      941793
556.0      800536
...
~~~

74. **Census_FirmwareVersionIdentifier**: no viene información en Kaggle sobre esta varaible, pero puede ser el identificador de la versión del firmware. Tiene alrededor de 50K valores únicos y 160133 missings (1.79%). No presenta correlación con la variable `Census_FirmwareManufacturerIdentifier`.

> `train['Census_FirmwareVersionIdentifier'].value_counts()`

~~~
33105.0    89611
33111.0    61583
33054.0    56626
33108.0    55040
11778.0    53785
....
~~~

75. **Census_IsSecureBootEnabled**: indica si el modo de arranque seguro está habilitado. Microsoft Secure Boot es un componente del sistema operativo Windows 8 de Microsoft que se basa en la funcionalidad de inicio seguro de la especificación UEFI para ayudar a evitar que se carguen aplicaciones de software malicioso y sistemas operativos "no autorizados" durante el proceso de inicio del sistema. Se ve como este criterio se reparte en 50-50 observaciones, por lo que nos puede parecer que es una variable determinante a la hora de decidir si el dispositivo se va a infectar o no.

> `train['Census_IsSecureBootEnabled'].value_counts()`

~~~
0    4585438
1    4336045
~~~

76. **~~Census_IsWIMBootEnabled~~**: no viene información de esta variable en Kaggle. WimBoot es una forma alternativa para que los OEM implementen Windows. Una implementación de WimBoot arranca y ejecuta Windows directamente desde un archivo de imagen comprimido de Windows (WIM). Este archivo WIM es inmutable y el acceso al mismo es administrado por un nuevo controlador de filtro del sistema de archivos (WoF.sys). El resultado es una reducción significativa en el espacio en disco requerido para instalar Windows. Tiene solo un único valor para (1), y 5659703 missings (63.44%), por lo que podemos prescindir de ella (https://docs.microsoft.com/en-us/windows/win32/w8cookbook/windows-image-file-boot--wimboot-).


> `train['Census_IsWIMBootEnabled'].value_counts()`

~~~
NA     5659703
0.0    3261779
1.0          1
~~~

77. **Census_IsVirtualDevice**: identifica si es una máquina virtual. Tiene 15953 missings (0.18%).
    
> `train['Census_IsVirtualDevice'].value_counts()`

~~~
0.0    8842840
1.0      62690
NA       15953
~~~

78. **Census_IsTouchEnabled**: responde a la pregunta de si es un dispositivo táctil, donde la mayoría (0) no lo son.
    
> `train['Census_IsTouchEnabled'].value_counts()`

~~~
0    7801452
1    1120031
~~~

> `train[['Census_IsTouchEnabled','HasDetections']].groupby(['Census_IsTouchEnabled','HasDetections']).size()`

~~~
Census_IsTouchEnabled  HasDetections
0                      0                3842617
                       1                3958835
1                      0                 619974
                       1                 500057
~~~

79. **Census_IsPenCapable**: responde a la pregunta de si el dispositivo es capaz de introducir el lápiz (relacionado un poco con si el dispositivo es táctil o no), donde la mayoría (0) no lo son.

> `train['Census_IsPenCapable'].value_counts()`

~~~
0    8581834
1     339649
~~~

~~~python
# Veamos la correlación que tiene con Census_IsTouchEnabled

aux.plot_corr(aux.corr(train, ['Census_IsPenCapable','Census_IsTouchEnabled']))
~~~

<img src="imagenes/Census_IsPenCapable.png" width=500 height=500 />    

80. **Census_IsAlwaysOnAlwaysConnectedCapable**: recupera información sobre si la batería permite que el dispositivo se conecte Always-On-Always-Connected. Por ejemplo, para especificar si el Wi-Fi debe permanecer encendido cuando la pantalla entra en reposo, se establece Always-On-Always-Connected (https://docs.microsoft.com/en-us/previous-versions/system-center/system-center-2012-R2/gg682088(v=technet.10)?redirectedfrom=MSDN). La variable vale 0 o DESACTIVADO si se desactiva el Wi-Fi cuando la pantalla se apaga. Para mantener la opción de Wi-Fi activado es en `Configuración > Wi-Fi> Manage`. La variable vale 1 o ACTIVADO si permite que el Wi-Fi esté siempre activado de manera predeterminada cuando la pantalla se apaga. Tiene 71343 missings	(0.80%).

> `train['Census_IsAlwaysOnAlwaysConnectedCapable'].value_counts()`
~~~
0.0    8341972
1.0     508168
NA       71343
~~~

81. **Wdft_IsGamer**: indica si el dispositivo es un dispositivo de jugador o no según su combinación de hardware, puesto que de normal las prestaciones de un ordenador gamer suelen ser muy altas. Tiene 303451 missings (3.40%).

> `train['Wdft_IsGamer'].value_counts()`

~~~
0.0    6174143
1.0    2443889
NA      303451
~~~

> `train[['Wdft_IsGamer', 'HasDetections']].groupby(['Wdft_IsGamer', 'HasDetections']).size()`

~~~
Wdft_IsGamer  HasDetections
0.0           0                3197504
              1                2976639
1.0           0                1119559
              1                1324330
~~~

82. **Wdft_RegionIdentifier**: no hay información sobre esta variable en Kaggle, tiene 15 valores únicos. La documentación de Microsoft menciona que es un identificador regional. Como no tiene casi correlación con otras variables, la dejamos. Tiene 303451 missings (3.40%).

> `train['Wdft_RegionIdentifier'].value_counts()`

~~~
10.0    1800105
11.0    1347828
3.0     1295892
1.0     1232258
15.0    1017591
7.0      597297
8.0      276029
13.0     225130
5.0      205372
12.0     163711
6.0      158163
4.0      135567
9.0       79882
2.0       79385
14.0       3822
~~~

~~~python
# Veamos la correlación con los demás identificadores de "regiones"

aux.plot_corr(aux.corr(train, ['Wdft_RegionIdentifier','CountryIdentifier',
                               'CityIdentifier','OrganizationIdentifier','GeoNameIdentifier']))
~~~

<img src="imagenes/Wdft_RegionIdentifier.png" width=500 height=500 />    

83. **HasDetections**: es la variable target, la que tenemos que predecir, y está partida en un 50% para ordenadores sin malware (0) y en otro 50% para ordenadores con malware (1).

> `train['HasDetections'].value_counts()`

~~~
0    4462591
1    4458892
~~~